In [1]:
import os
import json
import cv2

In [ ]:
"""
Downloader
"""

import os
import json
import cv2


def download(video_path, ytb_id, proxy=None):
    """
    ytb_id: youtube_id
    save_folder: save video folder
    proxy: proxy url, defalut None
    """
    if proxy is not None:
        proxy_cmd = "--proxy {}".format(proxy)
    else:
        proxy_cmd = ""
    if not os.path.exists(video_path):
        down_video = " ".join([
            "yt-dlp",
            proxy_cmd,
            '-f', "'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio'",
            '--skip-unavailable-fragments',
            '--merge-output-format', 'mp4',
            "https://www.youtube.com/watch?v=" + ytb_id, "--output",
            video_path, "--external-downloader", "aria2c",
            "--external-downloader-args", '"-x 16 -k 1M"'
        ])
        print(down_video)
        status = os.system(down_video)
        if status != 0:
            print(f"video not found: {ytb_id}")


def process_ffmpeg(raw_vid_path, save_folder, save_vid_name,
                   bbox, time):
    """
    raw_vid_path:
    save_folder:
    save_vid_name:
    bbox: format: top, bottom, left, right. the values are normalized to 0~1
    time: begin_sec, end_sec
    """

    def secs_to_timestr(secs):
        hrs = secs // (60 * 60)
        min = (secs - hrs * 3600) // 60 # thanks @LeeDongYeun for finding & fixing this bug
        sec = secs % 60
        end = (secs - int(secs)) * 100
        return "{:02d}:{:02d}:{:02d}.{:02d}".format(int(hrs), int(min),
                                                    int(sec), int(end))

    def expand(bbox, ratio):
        top, bottom = max(bbox[0] - ratio, 0), min(bbox[1] + ratio, 1)
        left, right = max(bbox[2] - ratio, 0), min(bbox[3] + ratio, 1)

        return top, bottom, left, right

    def to_square(bbox):
        top, bottom, leftx, right = bbox
        h = bottom - top
        w = right - leftx
        c = min(h, w) // 2
        c_h = (top + bottom) / 2
        c_w = (leftx + right) / 2

        top, bottom = c_h - c, c_h + c
        leftx, right = c_w - c, c_w + c
        return top, bottom, leftx, right

    def denorm(bbox, height, width):
        top, bottom, left, right = \
            round(bbox[0] * height), \
            round(bbox[1] * height), \
            round(bbox[2] * width), \
            round(bbox[3] * width)

        return top, bottom, left, right

    out_path = os.path.join(save_folder, save_vid_name)

    cap = cv2.VideoCapture(raw_vid_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    top, bottom, left, right = to_square(
        denorm(expand(bbox, 0.02), height, width))
    start_sec, end_sec = time

    cmd = f"ffmpeg -i {raw_vid_path} -vf crop=w={right-left}:h={bottom-top}:x={left}:y={top} -ss {secs_to_timestr(start_sec)} -to {secs_to_timestr(end_sec)} -loglevel error {out_path}"
    os.system(cmd)
    return out_path


def load_data(file_path):
    with open(file_path) as f:
        data_dict = json.load(f)

    for key, val in data_dict['clips'].items():
        save_name = key+".mp4"
        ytb_id = val['ytb_id']
        time = val['duration']['start_sec'], val['duration']['end_sec']

        bbox = [val['bbox']['top'], val['bbox']['bottom'],
                val['bbox']['left'], val['bbox']['right']]
        yield ytb_id, save_name, time, bbox


if __name__ == '__main__':
    json_path = 'celebvhq_info.json'  # json file path
    raw_vid_root = './downloaded_celebvhq/raw/'  # download raw video path
    processed_vid_root = './downloaded_celebvhq/processed/'  # processed video path
    proxy = None  # proxy url example, set to None if not use

    os.makedirs(raw_vid_root, exist_ok=True)
    os.makedirs(processed_vid_root, exist_ok=True)

    for vid_id, save_vid_name, time, bbox in load_data(json_path):
        raw_vid_path = os.path.join(raw_vid_root, vid_id + ".mp4")
        # Downloading is io bounded and processing is cpu bounded.
        # It is better to download all videos firstly and then process them via mutiple cpu cores.
        download(raw_vid_path, vid_id, proxy)
        # process_ffmpeg(raw_vid_path, processed_vid_root, save_vid_name, bbox, time)

    # with open('./ytb_id_errored.log', 'r') as f:
    #     lines = f.readlines()
    # for line in lines:
    #     raw_vid_path = os.path.join(raw_vid_root, line + ".mp4")
    #     download(raw_vid_path, line)

yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=M2Ohb0FAaJU --output ./downloaded_celebvhq/raw/M2Ohb0FAaJU.mp4 --external-downloader aria2c --external-downloader-args "-x 16 -k 1M"


[youtube] Extracting URL: https://www.youtube.com/watch?v=M2Ohb0FAaJU
[youtube] M2Ohb0FAaJU: Downloading webpage
[youtube] M2Ohb0FAaJU: Downloading ios player API JSON
[youtube] M2Ohb0FAaJU: Downloading mweb player API JSON
[youtube] M2Ohb0FAaJU: Downloading m3u8 information
[info] M2Ohb0FAaJU: Downloading 1 format(s): 137+140
[download] Destination: ./downloaded_celebvhq/raw/M2Ohb0FAaJU.f137.mp4
[download] 100% of   18.10MiB in 00:00:01 at 15.29MiB/s    
[download] Destination: ./downloaded_celebvhq/raw/M2Ohb0FAaJU.f140.m4a
[download] 100% of    2.07MiB in 00:00:00 at 6.30MiB/s   
[Merger] Merging formats into "./downloaded_celebvhq/raw/M2Ohb0FAaJU.mp4"
Deleting original file ./downloaded_celebvhq/raw/M2Ohb0FAaJU.f140.m4a (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/M2Ohb0FAaJU.f137.mp4 (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=_0t

[youtube] Extracting URL: https://www.youtube.com/watch?v=_0tf2n3rlJU
[youtube] _0tf2n3rlJU: Downloading webpage
[youtube] _0tf2n3rlJU: Downloading ios player API JSON
[youtube] _0tf2n3rlJU: Downloading mweb player API JSON
[youtube] _0tf2n3rlJU: Downloading m3u8 information
[info] _0tf2n3rlJU: Downloading 1 format(s): 137+140
[download] Destination: ./downloaded_celebvhq/raw/_0tf2n3rlJU.f137.mp4
[download] 100% of   54.96MiB in 00:00:04 at 11.12MiB/s    
[download] Destination: ./downloaded_celebvhq/raw/_0tf2n3rlJU.f140.m4a
[download] 100% of    7.52MiB in 00:00:01 at 7.03MiB/s   
[Merger] Merging formats into "./downloaded_celebvhq/raw/_0tf2n3rlJU.mp4"
Deleting original file ./downloaded_celebvhq/raw/_0tf2n3rlJU.f137.mp4 (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/_0tf2n3rlJU.f140.m4a (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=dvb

[youtube] Extracting URL: https://www.youtube.com/watch?v=dvbePi1o_Q0
[youtube] dvbePi1o_Q0: Downloading webpage
[youtube] dvbePi1o_Q0: Downloading ios player API JSON
[youtube] dvbePi1o_Q0: Downloading mweb player API JSON
[youtube] dvbePi1o_Q0: Downloading m3u8 information
[info] dvbePi1o_Q0: Downloading 1 format(s): 137+140
[download] Destination: ./downloaded_celebvhq/raw/dvbePi1o_Q0.f137.mp4
[download] 100% of   67.70MiB in 00:00:08 at 8.27MiB/s     
[download] Destination: ./downloaded_celebvhq/raw/dvbePi1o_Q0.f140.m4a
[download] 100% of    9.88MiB in 00:00:01 at 9.53MiB/s     
[Merger] Merging formats into "./downloaded_celebvhq/raw/dvbePi1o_Q0.mp4"
Deleting original file ./downloaded_celebvhq/raw/dvbePi1o_Q0.f137.mp4 (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/dvbePi1o_Q0.f140.m4a (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=e

[youtube] Extracting URL: https://www.youtube.com/watch?v=ekdWbe1s_d4
[youtube] ekdWbe1s_d4: Downloading webpage
[youtube] ekdWbe1s_d4: Downloading ios player API JSON
[youtube] ekdWbe1s_d4: Downloading mweb player API JSON
[youtube] ekdWbe1s_d4: Downloading m3u8 information
[info] ekdWbe1s_d4: Downloading 1 format(s): 137+140
[download] Destination: ./downloaded_celebvhq/raw/ekdWbe1s_d4.f137.mp4
[download] 100% of   31.18MiB in 00:00:03 at 8.54MiB/s     
[download] Destination: ./downloaded_celebvhq/raw/ekdWbe1s_d4.f140.m4a
[download] 100% of    2.03MiB in 00:00:00 at 7.01MiB/s   
[Merger] Merging formats into "./downloaded_celebvhq/raw/ekdWbe1s_d4.mp4"
Deleting original file ./downloaded_celebvhq/raw/ekdWbe1s_d4.f140.m4a (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/ekdWbe1s_d4.f137.mp4 (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=HV1

...

[youtube] Extracting URL: https://www.youtube.com/watch?v=uuTeJ6tbyB4
[youtube] uuTeJ6tbyB4: Downloading webpage
[youtube] uuTeJ6tbyB4: Downloading ios player API JSON
[youtube] uuTeJ6tbyB4: Downloading mweb player API JSON
[youtube] uuTeJ6tbyB4: Downloading m3u8 information
[info] uuTeJ6tbyB4: Downloading 1 format(s): 614+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 42
[download] Destination: ./downloaded_celebvhq/raw/uuTeJ6tbyB4.f614.mp4
[download] 100% of   19.03MiB in 00:00:05 at 3.55MiB/s                  
[download] Destination: ./downloaded_celebvhq/raw/uuTeJ6tbyB4.f140.m4a
[download] 100% of    3.11MiB in 00:00:00 at 10.65MiB/s    
[Merger] Merging formats into "./downloaded_celebvhq/raw/uuTeJ6tbyB4.mp4"
Deleting original file ./downloaded_celebvhq/raw/uuTeJ6tbyB4.f614.mp4 (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/uuTeJ6tbyB4.f140.m4a (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --ski

[youtube] Extracting URL: https://www.youtube.com/watch?v=uI8IKE4KNe4
[youtube] uI8IKE4KNe4: Downloading webpage
[youtube] uI8IKE4KNe4: Downloading ios player API JSON
[youtube] uI8IKE4KNe4: Downloading mweb player API JSON
[youtube] uI8IKE4KNe4: Downloading m3u8 information
[info] uI8IKE4KNe4: Downloading 1 format(s): 137+140
[download] Destination: ./downloaded_celebvhq/raw/uI8IKE4KNe4.f137.mp4
[download] 100% of   41.36MiB in 00:00:03 at 12.06MiB/s    
[download] Destination: ./downloaded_celebvhq/raw/uI8IKE4KNe4.f140.m4a
[download] 100% of    2.33MiB in 00:00:00 at 3.66MiB/s   
[Merger] Merging formats into "./downloaded_celebvhq/raw/uI8IKE4KNe4.mp4"
Deleting original file ./downloaded_celebvhq/raw/uI8IKE4KNe4.f140.m4a (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/uI8IKE4KNe4.f137.mp4 (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=Urp

[youtube] Extracting URL: https://www.youtube.com/watch?v=Urp9MjHLP0s
[youtube] Urp9MjHLP0s: Downloading webpage
[youtube] Urp9MjHLP0s: Downloading ios player API JSON
[youtube] Urp9MjHLP0s: Downloading mweb player API JSON
[youtube] Urp9MjHLP0s: Downloading m3u8 information
[info] Urp9MjHLP0s: Downloading 1 format(s): 137+140
[download] Destination: ./downloaded_celebvhq/raw/Urp9MjHLP0s.f137.mp4
[download] 100% of  121.42MiB in 00:00:19 at 6.26MiB/s     
[download] Destination: ./downloaded_celebvhq/raw/Urp9MjHLP0s.f140.m4a
[download] 100% of   11.93MiB in 00:00:02 at 4.65MiB/s   
[Merger] Merging formats into "./downloaded_celebvhq/raw/Urp9MjHLP0s.mp4"
Deleting original file ./downloaded_celebvhq/raw/Urp9MjHLP0s.f140.m4a (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/Urp9MjHLP0s.f137.mp4 (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=O56

[youtube] Extracting URL: https://www.youtube.com/watch?v=O56wZem2u3A
[youtube] O56wZem2u3A: Downloading webpage
[youtube] O56wZem2u3A: Downloading ios player API JSON
[youtube] O56wZem2u3A: Downloading mweb player API JSON
[youtube] O56wZem2u3A: Downloading m3u8 information
[info] O56wZem2u3A: Downloading 1 format(s): 137+140
[download] Destination: ./downloaded_celebvhq/raw/O56wZem2u3A.f137.mp4
[download] 100% of  269.10MiB in 00:00:45 at 5.92MiB/s     
[download] Destination: ./downloaded_celebvhq/raw/O56wZem2u3A.f140.m4a
[download] 100% of   15.78MiB in 00:00:01 at 12.08MiB/s    
[Merger] Merging formats into "./downloaded_celebvhq/raw/O56wZem2u3A.mp4"
Deleting original file ./downloaded_celebvhq/raw/O56wZem2u3A.f137.mp4 (pass -k to keep)
Deleting original file ./downloaded_celebvhq/raw/O56wZem2u3A.f140.m4a (pass -k to keep)
yt-dlp  -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio' --skip-unavailable-fragments --merge-output-format mp4 https://www.youtube.com/watch?v=O

[youtube] Extracting URL: https://www.youtube.com/watch?v=Op_fgLBxUQE
[youtube] Op_fgLBxUQE: Downloading webpage
[youtube] Op_fgLBxUQE: Downloading ios player API JSON
[youtube] Op_fgLBxUQE: Downloading mweb player API JSON
[youtube] Op_fgLBxUQE: Downloading m3u8 information
[info] Op_fgLBxUQE: Downloading 1 format(s): 614+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 32
[download] Destination: ./downloaded_celebvhq/raw/Op_fgLBxUQE.f614.mp4
[download]  59.6% of ~   8.25MiB at    1.89MiB/s ETA 00:02 (frag 18/32) 